In [ ]:
!pip install torch torchvision torchaudio

In [23]:
# import torch
# hf_model = torch.load('/u/mtian8/LLM/data/global_step17000/global_step17000_hugging_face/pytorch_model.bin')
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("/u/mtian8/LLM/data/global_step17000/global_step17000_hugging_face")
print(model)

/u/mtian8/anaconda3/envs/HF/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (no

In [28]:
show_state_dict_info(loaded["model"])

| Key | Value[0] |  
| --- | --- |  
|  decoder.final_layernorm.weight | ('Tensor', torch.Size([4096])) | 
|  decoder.layers.0.mlp.linear_fc1._extra_state | ('Object', <class '_io.BytesIO'>) | 
|  decoder.layers.0.mlp.linear_fc1.layer_norm_weight | ('Tensor', torch.Size([4096])) | 
|  decoder.layers.0.mlp.linear_fc1.weight | ('Tensor', torch.Size([22016, 4096])) | 
|  decoder.layers.0.mlp.linear_fc2._extra_state | ('Object', <class '_io.BytesIO'>) | 
|  decoder.layers.0.mlp.linear_fc2.weight | ('Tensor', torch.Size([4096, 11008])) | 
|  decoder.layers.0.self_attention.core_attention._extra_state | ('Object', <class '_io.BytesIO'>) | 
|  decoder.layers.0.self_attention.linear_proj._extra_state | ('Object', <class '_io.BytesIO'>) | 
|  decoder.layers.0.self_attention.linear_proj.weight | ('Tensor', torch.Size([4096, 4096])) | 
|  decoder.layers.0.self_attention.linear_qkv._extra_state | ('Object', <class '_io.BytesIO'>) | 
|  decoder.layers.0.self_attention.linear_qkv.layer_norm_weight |

In [32]:
loaded = torch.load('/work/nvme/bcbw/mtian8/model/MG_model/agpt17000_7B_tp_4_pp_1/iter_0000001/mp_rank_00/model_optim_rng.pt', map_location='cpu')

/tmp/ipykernel_3541587/2422443789.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load('/work/nvme/bcbw/mtian8/model/MG_model/agpt17000_7B_tp_4_pp_1/iter_

In [33]:
loaded['args']

Namespace(num_layers=32, encoder_num_layers=32, decoder_num_layers=None, hidden_size=4096, ffn_hidden_size=11008, num_attention_heads=32, kv_channels=128, group_query_attention=True, num_query_groups=8, max_position_embeddings=4096, position_embedding_type='rope', use_rotary_position_embeddings=True, rotary_base=10000, rotary_percent=1.0, rotary_interleaved=False, rotary_seq_len_interpolation_factor=None, add_position_embedding=False, make_vocab_size_divisible_by=128, normalization='RMSNorm', norm_epsilon=1e-05, apply_layernorm_1p=False, apply_residual_connection_post_layernorm=False, openai_gelu=False, squared_relu=False, swiglu=True, onnx_safe=None, bert_binary_head=True, untie_embeddings_and_output_weights=True, attention_dropout=0.1, hidden_dropout=0.1, weight_decay=0.01, start_weight_decay=0.01, end_weight_decay=0.01, weight_decay_incr_style='constant', clip_grad=1.0, adam_beta1=0.9, adam_beta2=0.999, adam_eps=1e-08, sgd_momentum=0.9, micro_batch_size=1, global_batch_size=1024, ra

In [1]:
from megatron.core import dist_checkpointing

def save_distributed_checkpoint(checkpoint_path, gpt_model):
    sharded_state_dict = gpt_model.sharded_state_dict(prefix='')
    dist_checkpointing.save(sharded_state_dict=sharded_state_dict, checkpoint_dir=checkpoint_path)

def load_distributed_checkpoint(checkpoint_path, gpt_model):
    sharded_state_dict=gpt_model.sharded_state_dict(prefix='')
    checkpoint = dist_checkpointing.load(sharded_state_dict=sharded_state_dict, checkpoint_dir=checkpoint_path)
    gpt_model.load_state_dict(checkpoint)
    return gpt_model



/projects/bcdz/mtian8/Megatron-LM/megatron/core/tensor_parallel/layers.py:278: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/projects/bcdz/mtian8/Megatron-LM/megatron/core/tensor_parallel/layers.py:294: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/projects/bcdz/mtian8/Megatron-LM/megatron/core/tensor_parallel/layers.py:389: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/projects/bcdz/mtian8/Megatron-LM/megatron/core/tensor_parallel/layers.py:428: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [ ]:
load_distributed_checkpoint

In [2]:
import torch
from collections import OrderedDict

# Path to the model checkpoint
checkpoint_paths = ['/u/mtian8/LLM/data/agpt/iter_0000140/mp_rank_00/model_optim_rng.pt', 
                    '/u/mtian8/LLM/data/agpt/iter_0000140/mp_rank_01/model_optim_rng.pt',
                    '/u/mtian8/LLM/data/agpt/iter_0000140/mp_rank_02/model_optim_rng.pt',
                    '/u/mtian8/LLM/data/agpt/iter_0000140/mp_rank_03/model_optim_rng.pt']

merged_state_dict = OrderedDict()


# Load the model checkpoint
# loaded = torch.load(checkpoint_path, map_location='cpu')

# Load the model checkpoints

original_state_dicts = {}

for checkpoint_path in checkpoint_paths:
    # Load the model checkpoint
    loaded = torch.load(checkpoint_path, map_location='cpu')
    state_dict = loaded["model"]
    original_state_dicts[checkpoint_path] = state_dict
    merged_state_dict.update(state_dict)
    # a more general way to merge state_dicts
    # for key in state_dict:
    #     if key not in merged_state_dict:
    #         merged_state_dict[key] = state_dict[key]
    #     else:
    #         merged_state_dict[key] += state_dict[key]




/tmp/ipykernel_3541587/763201809.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load(checkpoint_path, map_location='cpu')


KeyError: 'args'

In [19]:
def show_state_dict_info(*state_dicts):
    print("| Key | ", end="")
    for i, state_dict in enumerate(state_dicts):
        print(f"Value[{i}] | ", end=" ")
    print()
    print("| --- | ", end="")
    for i, state_dict in enumerate(state_dicts):
        print(f"--- | ", end=" ")
    print()
    def get_printable_content(value):
        if isinstance(value, torch.Tensor):
            return "Tensor", value.shape
        elif isinstance(value, list):
            return "List  ", len(value)
        else:
            return "Object", type(value)
    all_keys = set()
    for state_dict in state_dicts:
        all_keys.update(state_dict.keys())
    all_keys = sorted(list(all_keys))
    for key in all_keys: 
        print("| ", key, end=" | ")
        for state_dict in state_dicts:
            if key in state_dict:
                print(get_printable_content(state_dict[key]), end=" | ")
            else:
                print("None", end=" | ") 
        print()

In [20]:
show_state_dict_info(merged_state_dict, original_state_dicts[checkpoint_paths[0]])

| Key | Value[0] |  Value[1] |  
| --- | --- |  --- |  
|  decoder.final_layernorm.weight | ('Tensor', torch.Size([4096])) | ('Tensor', torch.Size([4096])) | 
|  decoder.layers.0.mlp.linear_fc1._extra_state | ('Object', <class '_io.BytesIO'>) | ('Object', <class '_io.BytesIO'>) | 
|  decoder.layers.0.mlp.linear_fc1.layer_norm_weight | ('Tensor', torch.Size([4096])) | ('Tensor', torch.Size([4096])) | 
|  decoder.layers.0.mlp.linear_fc1.weight | ('Tensor', torch.Size([5504, 4096])) | ('Tensor', torch.Size([5504, 4096])) | 
|  decoder.layers.0.mlp.linear_fc2._extra_state | ('Object', <class '_io.BytesIO'>) | ('Object', <class '_io.BytesIO'>) | 
|  decoder.layers.0.mlp.linear_fc2.weight | ('Tensor', torch.Size([4096, 2752])) | ('Tensor', torch.Size([4096, 2752])) | 
|  decoder.layers.0.self_attention.core_attention._extra_state | ('Object', <class '_io.BytesIO'>) | ('Object', <class '_io.BytesIO'>) | 
|  decoder.layers.0.self_attention.linear_proj._extra_state | ('Object', <class '_io.Byte

In [6]:
import pprint


pprint.pprint(list(original_state_dicts[checkpoint_paths[0]].keys()))

['embedding.word_embeddings.weight',
 'decoder.layers.0.self_attention.core_attention._extra_state',
 'decoder.layers.0.self_attention.linear_proj.weight',
 'decoder.layers.0.self_attention.linear_proj._extra_state',
 'decoder.layers.0.self_attention.linear_qkv.layer_norm_weight',
 'decoder.layers.0.self_attention.linear_qkv.weight',
 'decoder.layers.0.self_attention.linear_qkv._extra_state',
 'decoder.layers.0.mlp.linear_fc1.layer_norm_weight',
 'decoder.layers.0.mlp.linear_fc1.weight',
 'decoder.layers.0.mlp.linear_fc1._extra_state',
 'decoder.layers.0.mlp.linear_fc2.weight',
 'decoder.layers.0.mlp.linear_fc2._extra_state',
 'decoder.layers.1.self_attention.core_attention._extra_state',
 'decoder.layers.1.self_attention.linear_proj.weight',
 'decoder.layers.1.self_attention.linear_proj._extra_state',
 'decoder.layers.1.self_attention.linear_qkv.layer_norm_weight',
 'decoder.layers.1.self_attention.linear_qkv.weight',
 'decoder.layers.1.self_attention.linear_qkv._extra_state',
 'decod

In [43]:
args

Namespace(num_layers=32, encoder_num_layers=32, decoder_num_layers=None, hidden_size=4096, ffn_hidden_size=14336, num_attention_heads=32, kv_channels=128, group_query_attention=True, num_query_groups=8, max_position_embeddings=8192, position_embedding_type='rope', use_rotary_position_embeddings=True, rotary_base=10000, rotary_percent=1.0, rotary_interleaved=False, rotary_seq_len_interpolation_factor=None, add_position_embedding=False, make_vocab_size_divisible_by=128, normalization='RMSNorm', norm_epsilon=1e-05, apply_layernorm_1p=False, apply_residual_connection_post_layernorm=False, openai_gelu=False, squared_relu=False, swiglu=True, onnx_safe=None, bert_binary_head=True, untie_embeddings_and_output_weights=True, attention_dropout=0.1, hidden_dropout=0.1, weight_decay=0.01, start_weight_decay=0.01, end_weight_decay=0.01, weight_decay_incr_style='constant', clip_grad=1.0, adam_beta1=0.9, adam_beta2=0.999, adam_eps=1e-08, sgd_momentum=0.9, micro_batch_size=1, global_batch_size=1024, ra

In [45]:
args.rope_theta

AttributeError: 'Namespace' object has no attribute 'rope_theta'

In [26]:
loaded["model"]

OrderedDict([('embedding.word_embeddings.weight',
              tensor([[ 1.3733e-03,  5.0964e-03, -3.0365e-03,  ...,  2.2888e-03,
                       -1.9531e-03, -1.7166e-05],
                      [-2.7313e-03,  1.9379e-03, -1.3733e-03,  ..., -5.1498e-05,
                       -1.3962e-03, -1.9836e-03],
                      [ 9.5367e-04, -1.3367e-02,  4.1771e-04,  ...,  2.5940e-03,
                        7.0496e-03,  4.1809e-03],
                      ...,
                      [ 1.8715e-23,  3.2699e-24,  1.8198e-23,  ...,  5.3767e-23,
                       -2.2360e-24, -1.9852e-23],
                      [ 1.9335e-23, -1.8612e-24, -1.8818e-23,  ...,  2.3368e-23,
                        7.3412e-24, -3.1226e-23],
                      [-7.4860e-23, -6.3693e-23,  5.5059e-24,  ...,  4.9631e-24,
                       -5.4594e-23, -2.2877e-24]])),
             ('decoder.layers.0.self_attention.core_attention.fused_attention._extra_state',
              <_io.BytesIO at 0x151d8f234

In [29]:
for key in loaded["model"].keys():
    print(key)
    if hasattr(loaded["model"][key],"shape"):
        print('Shape: ',loaded["model"][key].shape)
    else:
        print(loaded["model"][key])
    print()

embedding.word_embeddings.weight
Shape:  torch.Size([128256, 4096])

decoder.layers.0.self_attention.core_attention.fused_attention._extra_state

decoder.layers.0.self_attention.linear_proj.weight
Shape:  torch.Size([4096, 4096])

decoder.layers.0.self_attention.linear_proj._extra_state

decoder.layers.0.self_attention.linear_qkv.layer_norm_weight
Shape:  torch.Size([4096])

decoder.layers.0.self_attention.linear_qkv.weight
Shape:  torch.Size([6144, 4096])

decoder.layers.0.self_attention.linear_qkv._extra_state

decoder.layers.0.mlp.linear_fc1.layer_norm_weight
Shape:  torch.Size([4096])

decoder.layers.0.mlp.linear_fc1.weight
Shape:  torch.Size([28672, 4096])

decoder.layers.0.mlp.linear_fc1._extra_state

decoder.layers.0.mlp.linear_fc2.weight
Shape:  torch.Size([4096, 14336])

decoder.layers.0.mlp.linear_fc2._extra_state

decoder.layers.1.self_attention.core_attention.fused_attention._extra_state

decoder.layers.1.self_attention.linear_proj.weight
Shape:  torch.Size([4096, 4096])

d

In [30]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("/home/tian/mtian8/LLM4Tutorial/model/HF_model/Meta-Llama-3-8B")
print(model)

Loading checkpoint shards: 100%|██████████| 4/4 [01:46<00:00, 26.58s/it]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
    (rotary_emb

In [33]:
for key, value in model.named_parameters():
    print(key, value.shape)
    # print()

model.embed_tokens.weight torch.Size([128256, 4096])
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 4096])
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.0.mlp.gate_proj.weight torch.Size([14336, 4096])
model.layers.0.mlp.up_proj.weight torch.Size([14336, 4096])
model.layers.0.mlp.down_proj.weight torch.Size([4096, 14336])
model.layers.0.input_layernorm.weight torch.Size([4096])
model.layers.0.post_attention_layernorm.weight torch.Size([4096])
model.layers.1.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 4096])
model.layers.1.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.1.mlp.gate_proj.weight torch.Size([14336, 4096])
model.layers.1.mlp.up_proj.weight torch.Size([14336, 4096])
model.

In [42]:
model.model.layers[0].self_attn.rotary_emb.__dict__

{'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('inv_freq',
               tensor([1.0000e+00, 8.1462e-01, 6.6360e-01, 5.4058e-01, 4.4037e-01, 3.5873e-01,
                       2.9223e-01, 2.3805e-01, 1.9392e-01, 1.5797e-01, 1.2869e-01, 1.0483e-01,
                       8.5397e-02, 6.9566e-02, 5.6670e-02, 4.6164e-02, 3.7606e-02, 3.0635e-02,
                       2.4955e-02, 2.0329e-02, 1.6560e-02, 1.3490e-02, 1.0990e-02, 8.9523e-03,
                       7.2927e-03, 5.9407e-03, 4.8394e-03, 3.9423e-03, 3.2114e-03, 2.6161e-03,
                       2.1311e-03, 1.7360e-03, 1.4142e-03, 1.1520e-03, 9.3847e-04, 7.6450e-04,
                       6.2277e-04, 5.0732e-04, 4.1327e-04, 3.3666e-04, 2.7425e-04, 2.2341e-04,
                       1.8199e-04, 1.4825e-04, 1.2077e-04, 9.8381e-05, 8.0143e-05, 6.5286e-05,
                       5.3183e-05, 4.3324e-05, 3.5292e-05, 2.8750e-05, 2.3420e-05, 1.9078e-05,
                       1.5542e-05, 1.2660e-05, 1.0313e-

In [2]:
from transformers import AutoModelForCausalLM
model_1 = AutoModelForCausalLM.from_pretrained("/home/tian/mtian8/LLM4Tutorial/model/HF_model/Mistral-7B-v0.1")

Loading checkpoint shards: 100%|██████████| 2/2 [00:58<00:00, 29.15s/it]


In [4]:
for key, value in model_1.named_parameters():
    print(key, value.shape)

model.embed_tokens.weight torch.Size([32000, 4096])
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 4096])
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.0.mlp.gate_proj.weight torch.Size([14336, 4096])
model.layers.0.mlp.up_proj.weight torch.Size([14336, 4096])
model.layers.0.mlp.down_proj.weight torch.Size([4096, 14336])
model.layers.0.input_layernorm.weight torch.Size([4096])
model.layers.0.post_attention_layernorm.weight torch.Size([4096])
model.layers.1.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 4096])
model.layers.1.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.1.mlp.gate_proj.weight torch.Size([14336, 4096])
model.layers.1.mlp.up_proj.weight torch.Size([14336, 4096])
model.l

In [1]:
# Compare two models
import torch
def diff(model1, model2):  
    for name1, param1 in model1.named_parameters():
        param2 = model2.state_dict()[name1]
        if not torch.equal(param1, param2):
            print(name1, "different by", torch.norm(param1 - param2).item())

In [2]:
from transformers import AutoModelForCausalLM
model_1 = AutoModelForCausalLM.from_pretrained("/home/tian/mtian8/LLM4Tutorial/model/HF_model/Codestral-22B-v0.1")
model_2 = AutoModelForCausalLM.from_pretrained("/home/tian/mtian8/LLM4Tutorial/model/HF_converted_model/Codestral-22B")


/home/tian/mtian8/anaconda3/envs/HF/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 19/19 [04:36<00:00, 14.56s/it]


In [3]:
diff(model_1, model_2)

In [10]:
for key, value in model_1.named_parameters():
    print(key, value.shape)

model.embed_tokens.weight torch.Size([32768, 6144])
model.layers.0.self_attn.q_proj.weight torch.Size([6144, 6144])
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 6144])
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 6144])
model.layers.0.self_attn.o_proj.weight torch.Size([6144, 6144])
model.layers.0.mlp.gate_proj.weight torch.Size([16384, 6144])
model.layers.0.mlp.up_proj.weight torch.Size([16384, 6144])
model.layers.0.mlp.down_proj.weight torch.Size([6144, 16384])
model.layers.0.input_layernorm.weight torch.Size([6144])
model.layers.0.post_attention_layernorm.weight torch.Size([6144])
model.layers.1.self_attn.q_proj.weight torch.Size([6144, 6144])
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 6144])
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 6144])
model.layers.1.self_attn.o_proj.weight torch.Size([6144, 6144])
model.layers.1.mlp.gate_proj.weight torch.Size([16384, 6144])
model.layers.1.mlp.up_proj.weight torch.Size([16384, 6144])
model.l

In [11]:
for key, value in model_2.named_parameters():
    print(key, value.shape)

model.embed_tokens.weight torch.Size([32768, 6144])
model.layers.0.self_attn.q_proj.weight torch.Size([6144, 6144])
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 6144])
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 6144])
model.layers.0.self_attn.o_proj.weight torch.Size([6144, 6144])
model.layers.0.mlp.gate_proj.weight torch.Size([16384, 6144])
model.layers.0.mlp.up_proj.weight torch.Size([16384, 6144])
model.layers.0.mlp.down_proj.weight torch.Size([6144, 16384])
model.layers.0.input_layernorm.weight torch.Size([6144])
model.layers.0.post_attention_layernorm.weight torch.Size([6144])
model.layers.1.self_attn.q_proj.weight torch.Size([6144, 6144])
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 6144])
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 6144])
model.layers.1.self_attn.o_proj.weight torch.Size([6144, 6144])
model.layers.1.mlp.gate_proj.weight torch.Size([16384, 6144])
model.layers.1.mlp.up_proj.weight torch.Size([16384, 6144])
model.l

In [2]:
import datasets
train_ds = datasets.load_from_disk('/lcrc/project/TuningLLMs/du/datasets/slimpajama_per_source_downsample/slimpajama_packed_32768_5b_per_source_down_sample_0.1.hf')
try:
    train_ds = train_ds['train']
except Exception:
    pass
train_ds = train_ds.with_format("np")

In [3]:
train_ds['input_ids'].shape

(149856, 32768)

In [1]:
import numpy as np
a =np.array([  654, 28649,  1413, 16969,   352, 24004,   271,  2751, 28705, 28781,
         28723, 28734, 28723, 28782,   325, 19658,  2383, 24142, 28725,  3885,
          6136, 28725, 15472, 28745,  3550,  1508,  2849, 28723,  4179,  2383,
         24142, 28723,   675, 28731,   354, 19492,   304,  5166,  6085, 16149])

In [13]:
a.shape

(40,)

In [11]:
train_ds['input_ids'][0][:40].shape

(40,)

In [16]:
for i in range(149856):
    print(i)
    # compare 2 numpy arrays are the same or not
    if np.array_equal(train_ds['input_ids'][i,:40], a):
        print("Found",i)
        print(train_ds['input_ids'][i][:40])
        break

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/tian/mtian8/LLM4Tutorial/model/HF_model/Mistral-7B-v0.1")

/home/tian/mtian8/anaconda3/envs/mg-lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer.decode(train_ds['input_ids'][0])

'<s> Can be done in the comfort of your own home!\nOn the verge of Diabetes and Heart Disease? Medical bills can be expensive. Act now!\nFast Training USA is a fitness center and one of the largest providers of privately labeled nutritional products in the United States! We offer our products to private business owners and trainers at an affordable price without any bulk orders.\nWe take an innovative approach on health & fitness. By combining our specially designed products that make you feel fuller faster while making you eat less, burn fat, speed metabolism, and block fat and carbs combined with personalized meal plans that get your body to strictly burn fat as it\'s only energy source, will make you lose weight in a safe, fast, healthy way. Additionally, we offer personal training to firm up the body and get you in the best shape of your life. Our in-house health consultants and personal trainers, will create a weight loss program for you that is guaranteed to help you lose the wei

In [6]:
import numpy as np
tokenizer.decode(np.array([  654, 28649,  1413, 16969,   352, 24004,   271,  2751, 28705, 28781,
         28723, 28734, 28723, 28782,   325, 19658,  2383, 24142, 28725,  3885,
          6136, 28725, 15472, 28745,  3550,  1508,  2849, 28723,  4179,  2383,
         24142, 28723,   675, 28731,   354, 19492,   304,  5166,  6085, 16149]))

'were analyzed using Mutation Surveyor version 4.0.5 (Softgenetics, State College, Pennsylvania; http://www.softgenetics.com) for alignment and multiple comparisons'

In [7]:
tokenizer.decode(np.array([ 5884, 26815, 19685,   272,  5587,   302, 15648, 28723, 10373,   396,
         7392,   466,   298,   272,  8712, 11806,  1307,   682,   835,  5297,
        15939,  1061,   294, 28748, 19701,  2867,   477,  5587,   264, 17661,
          356,   272,  1871,  5514,  1587,   302,   272,  1178,  3817, 28723]))

'thus permanently deny the setting of cookies. Such an adjustment to the Internet browser used would also prevent Automattic/Quantcast from setting a cookie on the information technology system of the data subject.'